<a href="https://colab.research.google.com/github/cawllol/NLP/blob/master/CombiningNLP_Models%26CustomRules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U spacy

     |████████████████████████████████| 10.0MB 2.7MB/s 
     |████████████████████████████████| 2.1MB 38.4MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [1]:
!pip install -U spacy-lookups-data

  Using cached https://files.pythonhosted.org/packages/41/b7/d5c635e51718c874606fb08249c0fab710286240d54847dc60bad3dfceac/spacy_lookups_data-0.3.2.tar.gz
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.3.2-py2.py3-none-any.whl size=93807572 sha256=263009f0ed9a72386a675ff5b1f58c7d51602e372ce452628e0c2005873c77fb
  Stored in directory: /root/.cache/pip/wheels/5b/f4/d0/bf720a06127c95d9be2a81d197a3f1998ee5fc63410944e28f
Successfully built spacy-lookups-data


In [2]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.1MB 962kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.0-cp36-none-any.whl size=12048607 sha256=e5c359efab7c0fb02bec1d8544759d573da0bf53e66dc4cb6c0758e10d756434
  Stored in directory: /tmp/pip-ephem-wheel-cache-lkgzp87s/wheels/be/ec/e4/29a68c7de525fcfa2d898cc27d2b945fe7cd966feeb2d85e0b
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
#Expanding named entities: Md Enamul Haque

In [6]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import  displacy

In [7]:
nlp=spacy.load('en_core_web_sm')

In [12]:
doc=nlp("Md. Enamul Haque chaired first board meeting at European Union")

In [13]:
doc

Md. Enamul Haque chaired first board meeting at European Union

In [14]:
#Entity name
print([(ent.text,ent.label_) for ent in doc.ents])

[('Enamul Haque', 'PERSON'), ('first', 'ORDINAL'), ('European Union', 'ORG')]


In [26]:
def add_title(doc):
  new_ents=[]
  for ent in doc.ents:
    if ent.label_=='PERSON' and ent.start!=0:
      prev_token=doc[ent.start-1]
      if prev_token.text in ('Dr.','Mr.','Md.'):
        new_ent=Span(doc,ent.start-1,ent.end,label=ent.label)
        new_ents.append(new_ent)
      else:
          new_ents.append(ent)
  doc.ents=new_ents
  return doc

In [27]:
nlp=spacy.load('en_core_web_sm')
nlp.add_pipe(add_title,after='ner')

In [28]:
doc=nlp("Md. Enamul Haque chaired first board meeting at European Union")

In [29]:
print([(ent.text,ent.label_) for ent in doc.ents])

[('Md. Enamul Haque', 'PERSON')]


In [30]:
#POS & dependency parsing
nlp=spacy.load('en_core_web_sm')

In [73]:
doc=nlp("Enamul Haque was meeting at European Union")

In [74]:
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

In [75]:
def get_person_orgs(doc):
  person_entities=[ent for ent in doc.ents if ent.label_=="PERSON"]
  for ent in person_entities:
    head=ent.root.head
    if head.lemma_=='work':
      preps=[token for token in head.children if token.dep_=='prep']
      for prep in preps:
        orgs=[token for token in prep.children if token.ent_type_=='ORG']
        print({'person':ent,'orgs':orgs,'past':head.tag_=="VBD"})
  return doc

In [76]:
from spacy.pipeline import  merge_entities

In [77]:
nlp=spacy.load('en_core_web_sm')

In [78]:
nlp.add_pipe(merge_entities)

In [79]:
nlp.add_pipe(get_person_orgs)

In [81]:
doc=nlp("Enamul Haque was meeting at European Union")